In [1]:
import cv2
import os
import numpy as np
import sqlite3

db = sqlite3.connect('/home/xilinx/capstone/database.sqlite3')
db.row_factory = sqlite3.Row
cur = db.cursor()
users = dict([(x['name'], dict(zip(('id', 'access_level'), (x['id'], x['access_level'])))) for x in cur.execute('''select * from users''').fetchall()])

def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow Haar classifier
    face_cascade = cv2.CascadeClassifier('/home/xilinx/capstone/lbpcascade_frontalface.xml')

    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

print("Preparing data...")
data_folder_path = "/home/xilinx/capstone/training"

#------STEP-1--------
#get the directories (one directory for each subject) in data folder
dirs = os.listdir(data_folder_path)

#list to hold all subject faces
faces = []
#list to hold labels for all subjects
labels = []

dirs = [d for d in dirs if d[0] != '.']

#let's go through each directory and read images within it
for dir_name in dirs:
    print(dir_name)
    #build path of directory containin images for current subject subject
    #sample subject_dir_path = "training-data/s1"
    subject_dir_path = data_folder_path + "/" + dir_name

    #get the images names that are inside the given subject directory
    subject_images_names = os.listdir(subject_dir_path)

    #------STEP-3--------
    #go through each image name, read image, 
    #detect face and add face to list of faces
    for image_name in subject_images_names:

        #ignore system files like .DS_Store
        if image_name.startswith("."):
            continue;
        print(image_name)
        #build image path
        #sample image path = training-data/s1/1.pgm
        image_path = subject_dir_path + "/" + image_name

        #read image
        image = cv2.imread(image_path)

        #display an image window to show the image 
        #cv2.imshow("Training on image...", cv2.resize(image, (400, 500)))
        #cv2.waitKey(100)

        #detect face
        face, rect = detect_face(image)

        #------STEP-4--------
        #for the purpose of this tutorial
        #we will ignore faces that are not detected
        if face is not None:
            #add face to list of faces
            faces.append(face)
            #add label for this face
            labels.append(users[dir_name]['id'])
        else:
            print(image_path)
            

print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

face_recognizer = cv2.face.LBPHFaceRecognizer_create()

face_recognizer.train(faces, np.array(labels))

for dir_name in dirs:
    face_recognizer.setLabelInfo(users[dir_name]['id'], dir_name)

face_recognizer.write("/home/xilinx/capstone/recognizer.dat")


Preparing data...
Rajan Patel
20190328_132336.jpg
20190328_132325.jpg
test4.jpg
20190328_132319.jpg
20190328_132322.jpg
20190328_132358.jpg
Jaynish Patel
51437f97df884acfba55008fd4c3d15b.jpeg
3e349ec777f744f9afa2098bec582f76.jpeg
d4a24b2f91d040a3a7d5c8c621051a26.jpeg
test3.jpg
7239c68bedfc40218e88fed966c41992.jpeg
Nikul Patel
test5.jpg
IMG2200870852229403207.jpg
IMG6564694865294845679.jpg
IMG4536445850074440756.jpg
IMG5124126177618914142.jpg
IMG7199662258560109910.jpg
Data prepared
Total faces:  17
Total labels:  17


In [2]:
print(face_recognizer.getLabelInfo(0))
print(face_recognizer.getLabelInfo(1))
print(face_recognizer.getLabelInfo(2))
print(face_recognizer.getLabelInfo(3))


Jaynish Patel
Rajan Patel
Nikul Patel


In [3]:
users

{'Jaynish Patel': {'access_level': 3, 'id': 1},
 'Nikul Patel': {'access_level': 1, 'id': 3},
 'Rajan Patel': {'access_level': 2, 'id': 2}}